In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import chi2
import nltk
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [ ]:
dfm = pd.read_csv('extracted_feature_values_midi.csv')
cluster_mood_filename = dfm.cluster_mood_filename
temp = []
cluster = []
mood = []
filename = []

for i in cluster_mood_filename:
    temp.append(i)
temp = [i.split('\\') for i in temp]

for path in temp:
    cluster.append(path[-3])
    mood.append(path[-2])
    filename.append(path[-1].split('.')[0])

dfm['cluster'] = cluster
dfm['mood'] = mood
dfm['filename'] = filename

clusters = dfm['cluster']
clusternumber = []

for cluster in clusters:
    temp = cluster.split(' ')
    clusternumber.append(temp[1])
dfm['cluster'] = clusternumber

dfm = dfm.fillna(0)

dfm.drop('cluster_mood_filename', axis=1, inplace=True)
dfm.drop('mood', axis=1, inplace=True)
#dfm.drop('cluster', axis=1, inplace=True)



dfm.head()

In [ ]:
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    # clean (convert to lowercase and remove punctuations and   
    # characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

In [ ]:
dfl = pd.read_csv("lyric_bag.csv")
lst_stopwords = nltk.corpus.stopwords.words("english")

dfl["text_clean"] = dfl["text"].apply(lambda x: 
          utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, 
          lst_stopwords=lst_stopwords))

In [ ]:
filenames = dfl['filename']
clusters = dfl['cluster']
clusternumber = []
filenumber = []

for file in filenames:
    temp = file.split('.')
    filenumber.append(temp[0])
dfl['filename'] = filenumber

for cluster in clusters:
    temp = cluster.split('r')
    clusternumber.append(temp[1])
dfl['cluster'] = clusternumber

dfl.drop('text', axis=1, inplace=True)
dfl.drop('mood_file', axis=1, inplace=True)
dfl.drop('mood', axis=1, inplace=True)
dfl.drop('cluster', axis=1, inplace=True)

dfl.head()

In [ ]:
df = dfm.merge(dfl,how='left',on='filename')
df.drop('filename', axis=1, inplace=True)
df = df.dropna()
df.reset_index(drop=True, inplace=True)
print(df.shape)

In [ ]:
df.head()

In [ ]:
Y = df['cluster']
Y = pd.DataFrame(Y)
X = df.drop('cluster', axis=1, inplace=False)
xcols = [col for col in X.columns]
midicols = xcols[:-1]
X_midi = X[midicols]

normalized_X = X_midi.values
std_scalor = preprocessing.MinMaxScaler()
X_scaled = std_scalor.fit_transform(normalized_X)
X_midi = pd.DataFrame(X_scaled, columns=midicols)

combinedX = X_midi
combinedX['text_clean'] = X['text_clean'].values

combinedX.isnull().values.any()

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(combinedX, Y, test_size=0.2, stratify=Y)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

In [ ]:
X_train2 = X_train.drop('text_clean', axis=1, inplace=False)
X_test2 = X_test.drop('text_clean', axis=1, inplace=False)

# param = {'C': [0.001,0.01,0.1, 1, 10],
#               'kernel': ['linear']}
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=20)
# grid = GridSearchCV(svm.SVC(), param, verbose = 3, cv=cv, return_train_score=True, n_jobs=15)
# grid.fit(X_train2, Y_train.values.ravel())
    
# train_accuracy_svm_norm = grid.best_score_
    
# grid_predict = grid.predict(X_test2)
# test_accuracy_svm_norm  = metrics.accuracy_score(Y_test,grid_predict)

# print("Training accuracy " + str(train_accuracy_svm_norm))
# print("Test accuracy " + str(test_accuracy_svm_norm))
# print(grid.best_params_)

# grid_predict = grid.predict(X_test2)

# print(metrics.classification_report(Y_test, grid_predict))

clf = svm.SVC(C=0.01, kernel='linear')

clf.fit(X_train2, Y_train.values.ravel())
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=20)

train_accuracy_svm_norm = cross_val_score(clf, X_train2, Y_train.values.ravel(), cv=cv, n_jobs=15).mean()
test_accuracy_svm_norm = clf.score(X_test2, Y_test.values.ravel())
print("Training accuracy " + str(train_accuracy_svm_norm))
print("Test accuracy " + str(test_accuracy_svm_norm))

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,3))
train_vector = vectorizer.fit_transform(X_train['text_clean'].values)
test_vector = vectorizer.transform(X_test['text_clean'].values)

print(train_vector.shape, test_vector.shape)
print(type(train_vector), type(test_vector))

In [ ]:
train_vector = pd.DataFrame(train_vector.todense(), columns=vectorizer.get_feature_names())
test_vector = pd.DataFrame(test_vector.todense(), columns=vectorizer.get_feature_names())
train_vector = train_vector.reset_index(drop=True)
test_vector = test_vector.reset_index(drop=True)
print(train_vector.shape, test_vector.shape)
print(type(train_vector), type(test_vector))
print(train_vector.isnull().values.any())
print(test_vector.isnull().values.any())

In [ ]:
%%time
# param = {'C': [0.001,0.01,0.1, 1, 10,100],
#               'kernel': ['linear']}

# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=20)

# grid = GridSearchCV(svm.SVC(), param, verbose = 3, cv=cv, return_train_score=True, n_jobs=15)
# grid.fit(train_vector, Y_train.values.ravel())
# train_accuracy_svm_lyr = grid.best_score_
# grid_predict = grid.predict(test_vector)
# test_accuracy_svm_lyr = metrics.accuracy_score(Y_test,grid_predict)
# print("Training accuracy " + str(train_accuracy_svm_lyr))
# print("Test accuracy " + str(test_accuracy_svm_lyr))
# print(grid.best_params_)

# grid_predict = grid.predict(test_vector)

# print(metrics.classification_report(Y_test, grid_predict))

clf = svm.SVC(C=10, kernel='linear')

clf.fit(train_vector, Y_train.values.ravel())
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=20)

train_accuracy_svm_lyr = cross_val_score(clf, train_vector, Y_train.values.ravel(), cv=cv, n_jobs=15).mean()
test_accuracy_svm_lyr = clf.score(test_vector, Y_test.values.ravel())
print("Training accuracy " + str(train_accuracy_svm_lyr))
print("Test accuracy " + str(test_accuracy_svm_lyr))

In [ ]:
X_train = X_train.drop('text_clean', axis=1, inplace=False)
X_test = X_test.drop('text_clean', axis=1, inplace=False)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)
print(X_train.isnull().values.any())
print(X_test.isnull().values.any())
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
train_vector = train_vector.reset_index(drop=True)
test_vector = test_vector.reset_index(drop=True)
X_train = pd.concat([X_train, train_vector], axis=1)
X_test = pd.concat([X_test, test_vector], axis=1)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)
print(X_train.isnull().values.any())
print(X_test.isnull().values.any())

In [ ]:
%%time
# param = {'C': [0.001,0.01,0.1, 1, 10],
#               'kernel': ['linear']}
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=20)
# grid = GridSearchCV(svm.SVC(), param, verbose = 3, cv=cv, return_train_score=True, n_jobs=15)
# grid.fit(X_train, Y_train.values.ravel())
    
# train_accuracy_svm_comb = grid.best_score_
    
# grid_predict = grid.predict(X_test)
# test_accuracy_svm_comb  = metrics.accuracy_score(Y_test,grid_predict)

# print("Training accuracy " + str(train_accuracy_svm_comb))
# print("Test accuracy " + str(test_accuracy_svm_comb))
# print(grid.best_params_)

# grid_predict = grid.predict(X_test)

# print(metrics.classification_report(Y_test, grid_predict))

clf = svm.SVC(C=0.01, kernel='linear', verbose = True)

clf.fit(X_train, Y_train.values.ravel())
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=20)

train_accuracy_svm_comb = cross_val_score(clf, X_train, Y_train.values.ravel(), cv=cv, n_jobs=15).mean()
test_accuracy_svm_comb = clf.score(X_test, Y_test.values.ravel())
print("Training accuracy " + str(train_accuracy_svm_comb))
print("Test accuracy " + str(test_accuracy_svm_comb))

In [ ]:
train_accuracy_list = [train_accuracy_svm_norm,train_accuracy_svm_lyr,train_accuracy_svm_comb]
test_accuracy_list = [test_accuracy_svm_norm,test_accuracy_svm_lyr,test_accuracy_svm_comb]
labels = ['Midi','Lyrics','Combined']

print(labels)
print(train_accuracy_list)
print(test_accuracy_list)

fig1,ax1 = plt.subplots()
ax1.set_title("Training Accuracy for SVM Classifier with feature selection")
ax1.bar(labels, train_accuracy_list)
plt.savefig("Train_Accuracy_cv.png")

fig2,ax2 = plt.subplots()
ax2.set_title("Testing Accuracy for SVM Classifier with feature selection")
ax2.bar(labels, test_accuracy_list, color='r')
plt.savefig("Test_Accuracy_cv.png")